In [2]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [29]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c40920d7a32340bfbf8b805e153928df = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OW62S_Lp5k2Z-g4lBAlWuaWoRBavHeEJoq1K-0b2pvZj',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c40920d7a32340bfbf8b805e153928df.get_object(Bucket='machinelearningwithpython-donotdelete-pr-bi8558lxmteiiq',Key='Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_pk = pd.read_csv(body)
df_pk.head()
body = client_c40920d7a32340bfbf8b805e153928df.get_object(Bucket='machinelearningwithpython-donotdelete-pr-bi8558lxmteiiq',Key='AssignmentData.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_pk = pd.read_csv(body)
df_pk.head()

,Borough,Avg-HousePrice,Longitude,Latitude,Population
0,东城区,104863,116.421885,39.938574,91900
1,西城区,115288,116.373190,39.934280,1243000
2,朝阳区,69621,116.521695,39.958953,3545000
3,丰台区,57295,116.258370,39.841938,2112000
4,石景山区,48417,116.184556,39.938867,616000


In [4]:
address = 'Beijing'

geolocator = Nominatim(user_agent="pk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Beijing are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Beijing are 39.906217, 116.3912757.


In [28]:
!wget --quiet https://geo.datav.aliyun.com/areas_v2/bound/110000_full.json -O beijing_districts.json
    
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [33]:
beijing_geo = r'beijing_districts.json' # geojson file


# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_pk['Avg-HousePrice'].min(),
                              df_pk['Avg-HousePrice'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# create a plain world map
beijing_chor = folium.Map([latitude, longitude], zoom_start=9)
beijing_chor.choropleth(
    geo_data=beijing_geo,
    data=df_pk,
    columns=['Borough', 'Avg-HousePrice'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average House Prices'
)

beijing_chor

In [38]:
map_beijing = folium.Map(location=[latitude, longitude], zoom_start=10)

In [37]:
for lat, lng, borough in zip(df_pk['Latitude'], df_pk['Longitude'], df_pk['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_beijing)
map_beijing

In [46]:
CLIENT_ID = 'XTN5F2O5QOIVLEE0VZMCSPTPB4PFTNQGQ4TSQ25C24YRJLBO' # your Foursquare ID
CLIENT_SECRET = 'VGD13EKLHDNLD10A5Y3PQWIE0MPSDLBU5LT31SOY0VKE5DKU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
beijing_venues = getNearbyVenues(names=df_pk['Borough'],
                                   latitudes=df_pk['Latitude'],
                                   longitudes=df_pk['Longitude']
                                  )
beijing_venues.head()

东城区
西城区
朝阳区
丰台区
石景山区
海淀区
门头沟区
房山区
通州区
顺义区
昌平区
大兴区
怀柔区
平谷区
密云区
延庆区


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,东城区,39.938574,116.421885,Traktirr Restaurant,39.941078,116.421306,Russian Restaurant
1,东城区,39.938574,116.421885,花开素食,39.939507,116.419882,Vegetarian / Vegan Restaurant
2,东城区,39.938574,116.421885,Ascott Raffles City 雅诗阁,39.938405,116.426129,Hotel
3,东城区,39.938574,116.421885,Baihe Vegetarian Restaurant (百合素食),39.941312,116.417877,Vegetarian / Vegan Restaurant
4,东城区,39.938574,116.421885,Capital Spirits (首都酒坊),39.937712,116.417098,Cocktail Bar


In [50]:
print(beijing_venues.shape)
beijing_venues.head()
beijing_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(beijing_venues['Venue Category'].unique())))

(44, 7)
There are 29 uniques categories.


In [51]:
# one hot encoding
beijing_onehot = pd.get_dummies(beijing_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
beijing_onehot['Neighborhood'] = beijing_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [beijing_onehot.columns[-1]] + list(beijing_onehot.columns[:-1])
beijing_onehot = beijing_onehot[fixed_columns]

beijing_onehot.head()

,Neighborhood,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cupcake Shop,Department Store,Dessert Shop,Fast Food Restaurant,Grocery Store,Hotel,Hotpot Restaurant,Japanese Restaurant,Korean Restaurant,Mexican Restaurant,Museum,New American Restaurant,Noodle House,Park,Pizza Place,Russian Restaurant,Sandwich Place,Shopping Mall,Vegetarian / Vegan Restaurant
0,东城区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,东城区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,东城区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,东城区,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,东城区,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
beijing_grouped = beijing_onehot.groupby('Neighborhood').mean().reset_index()
beijing_grouped

,Neighborhood,Asian Restaurant,Auto Garage,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Cupcake Shop,Department Store,Dessert Shop,Fast Food Restaurant,Grocery Store,Hotel,Hotpot Restaurant,Japanese Restaurant,Korean Restaurant,Mexican Restaurant,Museum,New American Restaurant,Noodle House,Park,Pizza Place,Russian Restaurant,Sandwich Place,Shopping Mall,Vegetarian / Vegan Restaurant
0,东城区,0.00,0.0,0.035714,0.00,0.000000,0.000000,0.035714,0.071429,0.035714,0.142857,0.035714,0.000000,0.035714,0.035714,0.035714,0.071429,0.035714,0.071429,0.035714,0.000000,0.000000,0.035714,0.000000,0.0,0.035714,0.107143,0.035714,0.035714,0.071429
1,昌平区,0.25,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000
2,朝阳区,0.00,0.5,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000
3,海淀区,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,西城区,0.00,0.0,0.000000,0.00,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [53]:
num_top_venues = 5

for hood in beijing_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = beijing_grouped[beijing_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----东城区----
                           venue  freq
0                    Coffee Shop  0.14
1             Russian Restaurant  0.11
2  Vegetarian / Vegan Restaurant  0.07
3             Chinese Restaurant  0.07
4            Japanese Restaurant  0.07


----昌平区----
                venue  freq
0    Asian Restaurant  0.25
1       Shopping Mall  0.25
2              Bakery  0.25
3  Chinese Restaurant  0.25
4   Hotpot Restaurant  0.00


----朝阳区----
              venue  freq
0              Park   0.5
1       Auto Garage   0.5
2  Asian Restaurant   0.0
3             Hotel   0.0
4     Shopping Mall   0.0


----海淀区----
                venue  freq
0  Chinese Restaurant   1.0
1    Asian Restaurant   0.0
2               Hotel   0.0
3       Shopping Mall   0.0
4      Sandwich Place   0.0


----西城区----
                venue  freq
0               Hotel  0.33
1            Beer Bar  0.11
2  Beijing Restaurant  0.11
3        Noodle House  0.11
4    Department Store  0.11




In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = beijing_grouped['Neighborhood']

for ind in np.arange(beijing_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(beijing_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,东城区,Coffee Shop,Russian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Japanese Restaurant,BBQ Joint,Cantonese Restaurant,Cocktail Bar,Cupcake Shop
1,昌平区,Asian Restaurant,Bakery,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Auto Garage,BBQ Joint,Beer Bar,Beijing Restaurant,Cantonese Restaurant
2,朝阳区,Auto Garage,Park,Vegetarian / Vegan Restaurant,Fast Food Restaurant,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant
3,海淀区,Chinese Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Auto Garage,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Cocktail Bar
4,西城区,Hotel,Department Store,Beer Bar,Noodle House,Beijing Restaurant,Museum,Mexican Restaurant,Vegetarian / Vegan Restaurant,Auto Garage,BBQ Joint


In [73]:
# set number of clusters
kclusters = 5

beijing_grouped_clustering = beijing_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(beijing_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

beijing_merged = df_pk

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
beijing_merged = beijing_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

beijing_merged.head(13) # check the last columns!

,Borough,Avg-HousePrice,Longitude,Latitude,Population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,东城区,104863,116.421885,39.938574,91900,3.0,Coffee Shop,Russian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Japanese Restaurant,BBQ Joint,Cantonese Restaurant,Cocktail Bar,Cupcake Shop
1,西城区,115288,116.373190,39.934280,1243000,0.0,Hotel,Department Store,Beer Bar,Noodle House,Beijing Restaurant,Museum,Mexican Restaurant,Vegetarian / Vegan Restaurant,Auto Garage,BBQ Joint
2,朝阳区,69621,116.521695,39.958953,3545000,2.0,Auto Garage,Park,Vegetarian / Vegan Restaurant,Fast Food Restaurant,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant
3,丰台区,57295,116.258370,39.841938,2112000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,石景山区,48417,116.184556,39.938867,616000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,海淀区,83895,116.239678,40.033162,3281000,1.0,Chinese Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Auto Garage,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Cocktail Bar
6,门头沟区,35387,115.795795,40.000893,290000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,房山区,25481,115.862836,39.726753,945000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,通州区,47173,116.740079,39.809815,1184000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,顺义区,39172,116.728229,40.154951,877000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
beijing_merged2 = beijing_merged.dropna()
beijing_merged2.head()

,Borough,Avg-HousePrice,Longitude,Latitude,Population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,东城区,104863,116.421885,39.938574,91900,3.0,Coffee Shop,Russian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Japanese Restaurant,BBQ Joint,Cantonese Restaurant,Cocktail Bar,Cupcake Shop
1,西城区,115288,116.373190,39.934280,1243000,0.0,Hotel,Department Store,Beer Bar,Noodle House,Beijing Restaurant,Museum,Mexican Restaurant,Vegetarian / Vegan Restaurant,Auto Garage,BBQ Joint
2,朝阳区,69621,116.521695,39.958953,3545000,2.0,Auto Garage,Park,Vegetarian / Vegan Restaurant,Fast Food Restaurant,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Chinese Restaurant
5,海淀区,83895,116.239678,40.033162,3281000,1.0,Chinese Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Auto Garage,BBQ Joint,Bakery,Beer Bar,Beijing Restaurant,Cantonese Restaurant,Cocktail Bar
10,昌平区,38836,116.216456,40.221724,1661000,4.0,Asian Restaurant,Bakery,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Auto Garage,BBQ Joint,Beer Bar,Beijing Restaurant,Cantonese Restaurant


In [78]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(beijing_merged2['Latitude'], beijing_merged2['Longitude'], beijing_merged2['Borough'], beijing_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
beijing_geo = r'beijing_districts.json' # geojson file


# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_pk['Avg-HousePrice'].min(),
                              df_pk['Avg-HousePrice'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

map_clusters.choropleth(
    geo_data=beijing_geo,
    data=df_pk,
    columns=['Borough', 'Avg-HousePrice'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average House Prices'
)

In [81]:
map_clusters